In [1]:
# !conda install -c anaconda seaborn -y

In [2]:
# conda activate smallcapcryptosenv

In [3]:
# !conda create --clone pyvizenv --name smallcapcryptosenv -y

In [4]:
# !pip install pycoingecko

In [5]:
# Initial Imports
import pandas as pd
import os
from pathlib import Path

import requests
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [6]:
def get_date_range(delta=365):
	datetime_2 = pd.Timestamp("today").normalize() # current date as timestamp
	datetime_1 = datetime_2 - pd.to_timedelta(delta, unit="Days")
	
	dates = {
		"date_1": [datetime_1,datetime_1.timestamp()],
		"date_2": [datetime_2,datetime_2.timestamp()]}
	return dates

In [7]:
def ticker_url(token_id, date_range):
	my_url = f"https://api.coingecko.com/api/v3/coins/{token_id}/market_chart/range?vs_currency=usd&from={date_range['date_1'][1]}&to={date_range['date_2'][1]}"
	return my_url

In [8]:
def get_json_from_api(url):
	return requests.get(url).json()

In [9]:
crypto_list = pd.read_csv(Path("./crypto_list.csv"))
crypto_list.drop(crypto_list.columns[-1],axis=1,inplace=True)

In [10]:
# btc_eth = pd.read_csv(Path("Resources/btc_eth.csv"))
# btc_eth.drop(btc_eth.columns[-1],axis=1,inplace=True)

In [11]:
# btc_eth

In [12]:
crypto_list.head()

,ticker,crypto_name,token_id
0,BTC,BITCOIN,bitcoin
1,ETH,ETHEREUM,ethereum
2,GFI,GOLDFINCH,goldfinch
3,PLU,PLUTON,pluton
4,UPI,PAWTOCOL,pawtocol


In [13]:
crypto_list.tail()

,ticker,crypto_name,token_id
40,LQTY,LIQUITY,liquity
41,BOND,BARNBRIDGE,barnbridge
42,TRB,TELLOR,tellor
43,ASM,ASSEMBLE PROTOCOL,assemble-protocol
44,NCT,POLYSWARM,polyswarm


### Coingecko API get request

In [14]:
my_dates = get_date_range() # this should only run once--keep out of loop

# my_dates['date_1'][0] # output (datetime):  Timestamp('2021-04-05 00:00:00')
# my_dates['date_1'][1] # output (unix time): 1617580800.0

In [15]:
# btc_dates = get_date_range(1500)
# btc_dates

    -> Run loop of get requests for cryptos

In [16]:
# loop for small cap cryptos ('crypto_list')
token_selection = [] # selecting only files with no zero values and 1 year of data available
i = 0
for i in range(len(crypto_list)):
	token_id = crypto_list['token_id'][i] # this should run within api call loop -- declare new token_id each api call from id_list
	url = ticker_url(token_id,my_dates) # this should run within api call loop -- new url each request
	crypto_json = get_json_from_api(url)
	
	df = pd.DataFrame.from_dict(crypto_json)
	raw_list = []
	
	j = 0
	for j in range(len(df['prices'])):
		raw = int(df['prices'][j][0]/1000)
		raw_list.append(raw)

		df['prices'][j] = df['prices'][j][1]
		df['market_caps'][j] = df['market_caps'][j][1]
		df['total_volumes'][j] = df['total_volumes'][j][1]
	
	df_dates = pd.to_datetime(raw_list, unit='s')
	
	df.insert(0,'date',df_dates)
	df.set_index('date',inplace=True)
	
	df['prices'] = df['prices'].astype(float, errors = 'raise')
	df['market_caps'] = df['market_caps'].astype(float, errors = 'raise')
	df['total_volumes'] = df['total_volumes'].astype(float, errors = 'raise')

	if 0 in df.values:
		print(f"{token_id} data not saved - zero values found in data")
	elif len(df)-1  < 365:
		print(f"{token_id} data not saved - {len(df)-1} days of data available (< 1 year)")
	else:
		token_selection.append(token_id)
		ticker_path = 'Data/' + token_id + '.csv'
		df.to_csv(ticker_path)

goldfinch data not saved - zero values found in data
cryptex-finance data not saved - zero values found in data
farmers-only data not saved - zero values found in data
ampleforth-governance-token data not saved - zero values found in data
loom-network-new data not saved - zero values found in data
gitcoin data not saved - zero values found in data
adventure-gold data not saved - 220 days of data available (< 1 year)
clover-finance data not saved - zero values found in data
kyber-network-crystal data not saved - zero values found in data
moss-carbon-credit data not saved - zero values found in data
orca data not saved - 243 days of data available (< 1 year)
wrapped-centrifuge data not saved - zero values found in data
luna-wormhole data not saved - zero values found in data
gods-unchained data not saved - zero values found in data
shping data not saved - zero values found in data
musd data not saved - zero values found in data
assemble-protocol data not saved - zero values found in data

In [17]:
print("CoinGecko API requests complete for all items in crypto_list.csv.")
print("Numerical data changed from dtype object to float.")
print(f"{len(token_selection)} csv files saved to Data folder.")

CoinGecko API requests complete for all items in crypto_list.csv.
Numerical data changed from dtype object to float.
28 csv files saved to Data folder.


In [25]:
crypto_subset = pd.DataFrame(token_selection)
crypto_subset = crypto_subset.transpose()

In [20]:
expand_date_range = False

if expand_date_range == True:
    # loop for bitcoin and ethereum (running separately with expanded date range)
    i = 0
    for i in range(len(btc_eth)):
        token_id = btc_eth['token_id'][i] # this should run within api call loop -- declare new token_id each api call from id_list
        url = ticker_url(token_id,btc_dates) # this should run within api call loop -- new url each request
        crypto_json = get_json_from_api(url)

        df = pd.DataFrame.from_dict(crypto_json)
        raw_list = []

        j = 0
        for j in range(len(df['prices'])):
            raw = int(df['prices'][j][0]/1000)
            raw_list.append(raw)

            df['prices'][j] = df['prices'][j][1]
            df['market_caps'][j] = df['market_caps'][j][1]
            df['total_volumes'][j] = df['total_volumes'][j][1]

        df_dates = pd.to_datetime(raw_list, unit='s')

        df.insert(0,'date',df_dates)
        df.set_index('date',inplace=True)

        df['prices'] = df['prices'].astype(float, errors = 'raise')
        df['market_caps'] = df['market_caps'].astype(float, errors = 'raise')
        df['total_volumes'] = df['total_volumes'].astype(float, errors = 'raise')

        ticker_path = 'Data/' + token_id + '.csv'
        df.to_csv(ticker_path)
else:
    print("Using same date range for Ethereum and Bitcoin")

Using same date range for Ethereum and Bitcoin


In [26]:
crypto_subset.to_csv('token_selection.csv', index=False)
print("token_selection.csv saved to root directory.")

token_selection.csv saved to root directory.
